<a href="https://colab.research.google.com/github/ander-yamamoto/dio-bairesdev/blob/main/face-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
# 1. Imports e Configurações
# =========================================
%matplotlib inline

import os

!pip install simple-image-download tqdm

from simple_image_download import simple_image_download as simp
from tqdm import tqdm
import requests

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.


In [2]:
# =========================================
# 2. Função para baixar imagens
# =========================================
def download_with_progress(group, query, num_images):
    downloader = simp.simple_image_download()
    links = downloader.urls(query, num_images)  # pega URLs sem baixar ainda

    save_dir = os.path.join(group, query)
    os.makedirs(save_dir, exist_ok=True)

    for url in tqdm(links, desc=f"Baixando {query}"):
        try:
            img_data = requests.get(url, timeout=5).content
            file_name = os.path.join(save_dir, os.path.basename(url.split("?")[0]))
            with open(file_name, "wb") as f:
                f.write(img_data)
        except:
            pass  # ignora erros

In [3]:
# =========================================
# 3. Baixar Dataset
# =========================================

group = "Friends"
category=["Ross", "Chandler", "Joey", "Phoebe", "Monica", "Rachel"]


for cat in category:
  if not os.path.exists(os.path.join(group, cat)):
    download_with_progress(group, cat, 20)

  else:
    print(f"Banco de imagens de {cat} já existe. Pulando download.")

# Listar categorias existentes

categories = [
    os.path.join(group, c)
    for c in os.listdir(group)
    if os.path.isdir(os.path.join(group, c))
]
print("Categorias:", categories)

Baixando Rachel: 100%|██████████| 20/20 [00:02<00:00,  7.80it/s]

Categorias: ['Friends/Joey', 'Friends/Ross', 'Friends/Phoebe', 'Friends/Rachel', 'Friends/Chandler', 'Friends/Monica']


In [4]:
!pip install MTCNN

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

from mtcnn import MTCNN

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {device}')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
Running on device: cuda:0


In [14]:
def crop_face_and_save(path, new_path=None, model=MTCNN, transformer=None, params=None):
    """
    Detect face on each image, crop them and save to "new_path"
    :param str path: path with images will be passed to  datasets.ImageFolder
    :param str new_path: path to locate new "aligned" images, if new_path is None
                     then new_path will be path + "_cropped"
    :param model: model to detect faces, default MTCNN
    :param transformer: transformer object will be passed to ImageFolder
    :param params: parameters of MTCNN model
    """
    if not new_path:
        new_path = path + '_cropped'

    # it is default parameters for MTCNN
    if not params:
        params = {
            'image_size': 160, 'margin': 0,
            'min_face_size': 10, 'thresholds': [0.6, 0.7, 0.7],
            'factor': 0.709, 'post_process': False, 'device': device
            }

    model = model(**params)

    if not transformer:
        transformer = transforms.Lambda(
            lambd=lambda x: x.resize((1280, 1280)) if (np.array(x) > 2000).all() else x
        )
    # for convenience we will use ImageFolder instead of getting Image objects by file paths
    dataset = datasets.ImageFolder(path, transform=transformer)
    dataset.samples = [(p, p.replace(path, new_path)) for p, _ in dataset.samples]

    # batch size 1 as long as we havent exact image size and MTCNN will raise an error
    loader = DataLoader(dataset, batch_size=1, collate_fn=training.collate_pil)
    for i, (x, y) in enumerate(tqdm.tqdm(loader)):
        model(x, save_path=y)

    # spare some memory
    del model, loader, dataset

In [15]:
crop_face_and_save(group)

TypeError: MTCNN.__init__() got an unexpected keyword argument 'image_size'

In [5]:
model=MTCNN
path = group
new_path = f'{group}__cropped'

transformer = transforms.Lambda(
            lambd=lambda x: x.resize((1280, 1280)) if (np.array(x) > 2000).all() else x
        )
dataset = datasets.ImageFolder(path, transform=transformer)
dataset.samples = [(p, p.replace(path, new_path)) for p, _ in dataset.samples]
loader = DataLoader(dataset, batch_size=1)
for i, (x, y) in enumerate(tqdm(np.array(loader))):
  model(x, save_path=y)

0it [00:00, ?it/s]


TypeError: iteration over a 0-d array

In [6]:
dataset = datasets.ImageFolder(path)

In [8]:
dataset.samples = [(p, p.replace(path, new_path)) for p, _ in dataset.samples]

In [11]:
loader = DataLoader(dataset, batch_size=1)

In [14]:
print(loader)